# Docker

## Что такое вирутальная машина? Что такое контейнер?

Стандарт в индустрии на сегодняшний день — это использовать виртуальные машины для запуска приложений. Виртуальные машины запускают приложения внутри гостевой операционной системы, которая работает на виртуальном железе основной операционной системы сервера.


Виртуальные машины отлично подходят для полной изоляции процесса для приложения: почти никакие проблемы основной операционной системы не могут повлиять на софт гостевой ОС, и наоборот. Но за такую изоляцию приходится платить. Существует значительная вычислительная нагрузка, необходимая для виртуализации железа гостевой ОС.


Контейнеры используют другой подход: они предоставляют схожий с виртуальными машинами уровень изоляции, но благодаря правильному задействованию низкоуровневых механизмов основной операционной системы делают это с в разы меньшей нагрузкой.

### Схема работы докера

<img src="https://docs.docker.com/engine/images/architecture.svg">

Терминология: 

* Images (образы) - Схемы нашего приложения, которые являются основой контейнеров.
* Containers (контейнеры) - Создаются на основе образа и запускают само приложение. Kонтейнер создаются командой docker run. Список запущенных контейнеров можно увидеть с помощью команды docker ps.
* Docker Daemon (демон Докера) - Фоновый сервис, запущенный на хост-машине, который отвечает за создание, запуск и уничтожение Докер-контейнеров. Демон — это процесс, который запущен на операционной системе, с которой взаимодействует клиент.
* Docker Client (клиент Докера) - Утилита командной строки, которая позволяет пользователю взаимодействовать с демоном. Существуют другие формы клиента, например, Kitematic, с графическим интерфейсом.
* Docker Hub - Регистр Докер-образов. Грубо говоря, архив всех доступных образов. Если нужно, то можно содержать собственный регистр и использовать его для получения образов.

## Создание своих образов: dockerfile

Dockerfile — это простой текстовый файл, в котором содержится список команд Докер-клиента. Это простой способ автоматизировать процесс создания образа.  Команды в Dockerfile почти идентичны своим аналогам в Linux. Это значит, что в принципе не нужно изучать никакой новый синтаксис, чтобы начать работать с докерфайлами.

### [Пример dockerfile](https://github.com/jupyter/docker-stacks/blob/master/docs/using/recipes.md)  

``` dockerfile

FROM jupyter/datascience-notebook:9f9e5ca8fe5a

# Install from requirements.txt file
COPY --chown=${NB_UID}:${NB_GID} requirements.txt /tmp/

RUN pip install --requirement /tmp/requirements.txt && \
    fix-permissions $CONDA_DIR && \
    fix-permissions /home/$NB_USER
    
```

Чтобы собрать этот образ можно воспользоваться следующей командой, если вы находитесь в текущей директории (аргументом -t задается имя и опционально тэги):

``` bash
docker build -t my-notebook .
```

Docker демон выполняет инструкции в Dockerfile одну за другой, производя коммит результата каждой инструкции в новый образ если это необходимо, перед тем как вывести финальный ID вашего нового образа.

### Инструкция ```FROM``` 

``` dockerfile
FROM [--platform=<platform>] <image> [AS <name>]
```

или

``` dockerfile
FROM [--platform=<platform>] <image>[:<tag>] [AS <name>]
```

или

``` dockerfile
FROM [--platform=<platform>] <image>[@<digest>] [AS <name>]
```


Инструкция ```FROM``` задает базовый образ для последующих инструкций. Dockerfile обязательно должен иметь инструкцию ```FROM```. Можно использовать любой работающий образ, проще всего начать с загрузки образа из публичного репозитория.

* ```FROM``` должен быть первой инструкцией в Dockerfile (не считая комментариев и директив парсера ```ARG```).
* ```FROM``` может использоваться несколько раз в пределах одного Dockerfile для создания нескольких образов или использования одного этапа сборки в качестве зависимости для другого.
* При желании можно дать имя новому этапу сборки, добавив имя AS к инструкции FROM. Имя можно использовать в последующих инструкциях ```FROM``` и ```COPY --from = <name>``` для ссылки на образ, созданный на этом этапе.
* Необязательный флаг ```--platform``` можно использовать для указания платформы изображения в случае, если FROM ссылается на многоплатформенный образ.
* Значения тега ```<tag>``` или дайджеста ```<digest>``` необязательны. Если вы опустите любой из них, конструктор по умолчанию использует последний тег. 

### Инструкция ```MAINTAINER```

Инструкция ```MAINTAINER``` используется для указания автора сборки.  

``` dockerfile 
MAINTAINER <name>
```

### Инструкция ```RUN```

Команда ```RUN``` имеет две формы:

* 
``` dockerfile
# shell form
RUN <command>
```
Команда выполняется в оболочке ```shell```, по умолчанию ```/bin/sh -c``` для Linux или ```cmd /S /C``` для Windows).
* 
``` dockerfile
# exec form
RUN ["executable", "param1", "param2"] 
```
exec форма выполнения команд позволяет разбивать строку команды и выполнять команды используя базовый образ, который не имеет исполняемого файла оболочки.

Инструкция RUN выполняет любые команды в новом слое поверх текущего образа и делает коммит результата. Полученный после коммита образ будет использован для следующего шага в Dockerfile.

Создание слоев инструкцией RUN и последующий их коммит является базовой концепцией Docker, которая позволяет создать контейнер из любой точки истории образа, по аналогии с системами контроля версий.

В ```shell``` форме вы можете использовать ```\``` (обратный слеш) в инструкциях RUN для переноса команды на следующую строку:

``` dockerfile
RUN /bin/bash -c 'source $HOME/.bashrc; \
echo $HOME'
```

Вместе они эквивалентны строке:

``` dockerfile
RUN /bin/bash -c 'source $HOME/.bashrc; echo $HOME'
```

Для использование другой оболочки отличной от ```/bin/sh```, используйте exec форму выполнения команды ```RUN```:

``` dockerfile
RUN ["/bin/bash", "-c", "echo hello"]
```

Кэш для инструкций ```RUN``` остается нетронутым до следующей сборки. Например кэш для инструкции ```RUN apt-get dist-upgrade -y``` будет повторно использован при следующей сборке (что за флаг ```-y```?). Кэш инструкции RUN может быть сброшен флагом ```--no-cache```, к примеру ```docker build --no-cache```.

### Инструкция ```CMD```

Инструкция ```CMD``` имеет три формы:

* 
``` dockerfile
CMD ["executable","param1","param2"]
```
exec форма, является предпочтительной.

* 
``` dockerfile
CMD ["param1","param2"]
```
в качестве параметров по умолчанию для ```ENTRYPOINT```

* 
``` dockerfile
CMD command param1 param2
``` 
shell форма

Инструкция ```CMD``` может быть использована только один раз в Dockerfile. Если вы используете больше одной ```CMD```, то только последняя инструкция будет работать.

Основное предназначение ```CMD``` передача параметров по-умолчанию для запуска контейнера. Эти значения по умолчанию могут включать в себя исполняемый файл, или же они могут опустить исполняемый файл, но в этом случае вы должны использовать инструкцию ```ENTRYPOINT```.

При использовании shell или exec форматов, инструкция ```CMD``` задает команду которая будет выполнена при запуске образа.


Если вы используете shell форму инструкции CMD, то команда ```<command>``` будет выполнена в ```/bin/sh -c```:

``` dockerfile
FROM ubuntu
CMD echo "This is a test."
``` 

Если вы хотите запустить команду ```<command>``` без оболочки, то вы должны написать команду в формате ```JSON``` массива и указать полный путь к исполняемому файлу. Этот формат является предпочтительным для ```CMD```. Любые дополнительные параметры должны быть отдельно перечислены в массиве:

``` dockerfile
FROM ubuntu
CMD ["/usr/bin/wc","--help"]
```


### Инструкция ```LABEL```

``` dockerfile
LABEL <key>=<value> <key>=<value> <key>=<value> ...
```

ИнструкцияLABEL добавляет метаданные для образа. LABEL состоит из пар ключ-значение. Для использования пробелов в значениях LABEL, используйте кавычки и обратный слеш как если бы вы находились в командной смтроке. Несколько примеров:

``` dockerfile
LABEL "com.example.vendor"="ACME Incorporated"
LABEL com.example.label-with-value="foo"
LABEL version="1.0"
LABEL description="This text illustrates \
that label-values can span multiple lines."
```

### Инструкция ```EXPOSE```

``` dockerfile
EXPOSE <port> [<port>...]
```

Инструкция ```EXPOSE``` указывает Docker, что контейнер слушает определенные порты после запуска. ```EXPOSE``` не делает порты контейнера доступными для хоста. Для этого, вы должны использовать флаг -p (что бы открыть диапазон портов) или флаг ```-P``` что бы открыть все порты из ```EXPOSE```. Можно задать один номер порта и пробросить его на другой внешний порт.

### Инструкция ```ENV```

``` dockerfile
ENV <key> <value>
ENV <key>=<value> ...
```

Инструкция ENV задает переменные окружения с именем <key> и значением <value>. Это значение будет находиться в окружении всех команд потомков Dockerfile и могут быть использованы как обычные переменные окружения.


Инструкция ENV имеет две формы. Первая форма, ENV <key> <value>, устанавливает значение одной переменной. Вся строка после первого пробела будет рассматриваться как <value> - включая пробелы и кавычки.

``` dockerfile
ENV myName="John Doe" myDog=Rex\ The\ Dog \
myCat=fluffy
```

    
Вторая форма, ENV <key>=<value> ..., позволяет задать сразу несколько переменных. Обратите внимание, что вторая форма использует в синтаксисе знак равенства (=), в то время как для первой формы это не нужно. Как и в случае разбора командной строки, ковычки и обратные слеши могут быть использованы для включения пробелов в значениях.
    
 
 ``` dockerfile   
ENV myName John Doe
ENV myDog Rex The Dog
ENV myCat fluffy
    ```
Оба примера приведут к одному результату в контейнере, но первый вариант предпочтительней, поскольку он создаст только один слой.

### Инструкция ```ADD```

Инструкция ```ADD``` копирует новые файлы, папки или удаленные файлы по URLs из ```<src>``` и добавляет их в файловую систему контейнера в ```<dest>```.

Возможно задать несколько <src> путей, но путь должен находится внутри контекста сборки.

``` dockerfile 
ADD <src>... <dest>
ADD ["<src>",... "<dest>"] 
``` 
Вторая форма используется для путей с пробелами. 

Все файлы и папки создаются с UID и GID равными 0.

### Инструкция ```COPY```

Инструкция COPY копирует новые файлы или каталоги из <src> и добавляет их в файловую систему контейнера в <dest>.
    
    
``` dockerfile 
COPY <src>... <dest>
COPY ["<src>",... "<dest>"] 
``` 
Возможно задать несколько <src> путей, но путь должен находится внутри контекста сборки.

    

### Инструкция ```VOLUME```

Инструкция ```VOLUME``` создает точку монтирования с заданным именем и помечает его как внешний смонтированный том из базового хоста или контейнера. Можно использовать JSON массив, ```VOLUME ["/var/log/"]```, или текстовую строку с несколькими аргументами, например VOLUME ```/var/log``` или VOLUME ```/var/log /var/db```.


``` dockerfile 
VOLUME ["/data"]
``` 


Еще несколько примеров dockerfile-ов:


### Nginx
``` dockerfile 

FROM      ubuntu
MAINTAINER Victor Vieux <victor@docker.com>

LABEL Description="This image is used to start the foobar executable" Vendor="ACME Products" Version="1.0"
RUN apt-get update && apt-get install -y inotify-tools nginx apache2 openssh-server
```

### Jupyter notebook with TF

``` dockerfile 
FROM tensorflow/tensorflow:2.3.0rc2-gpu-jupyter

RUN apt-get update && apt-get install -y --no-install-recommends \
        build-essential \
        curl wget apt libcairo2

COPY requirements.txt /tmp/
RUN pip3 install -r /tmp/requirements.txt

RUN pip3 install wandb
ADD seminars /seminars
WORKDIR seminars

CMD bash -c 'source /etc/bash.bashrc && jupyter notebook --notebook-dir=/seminars --ip 0.0.0.0 --no-browser --allow-root'


```

### Firefox over VNC

``` dockerfile 
FROM ubuntu

# Install vnc, xvfb in order to create a 'fake' display and firefox
RUN apt-get update && apt-get install -y x11vnc xvfb firefox
RUN mkdir ~/.vnc
# Setup a password
RUN x11vnc -storepasswd 1234 ~/.vnc/passwd
# Autostart firefox (might not be the best way, but it does the trick)
RUN bash -c 'echo "firefox" >> /.bashrc'

EXPOSE 5900
CMD    ["x11vnc", "-forever", "-usepw", "-create"]
```


Мы разобрали не все инструкции dockerfile. Полный список вы можете найти [здесь](https://docs.docker.com/engine/reference/builder/).

## Работа с готовыми образами и контейнерами:

### Список возможных команд:

``` bash
$ docker help

Usage:	docker [OPTIONS] COMMAND

A self-sufficient runtime for containers

Options:
      --config string      Location of client config files (default
                           "/home/tv-home/.docker")
  -c, --context string     Name of the context to use to connect to the
                           daemon (overrides DOCKER_HOST env var and
                           default context set with "docker context use")
  -D, --debug              Enable debug mode
  -H, --host list          Daemon socket(s) to connect to
  -l, --log-level string   Set the logging level
                           ("debug"|"info"|"warn"|"error"|"fatal")
                           (default "info")
      --tls                Use TLS; implied by --tlsverify
      --tlscacert string   Trust certs signed only by this CA (default
                           "/home/tv-home/.docker/ca.pem")
      --tlscert string     Path to TLS certificate file (default
                           "/home/tv-home/.docker/cert.pem")
      --tlskey string      Path to TLS key file (default
                           "/home/tv-home/.docker/key.pem")
      --tlsverify          Use TLS and verify the remote
  -v, --version            Print version information and quit
```

```
Management Commands:
  builder     Manage builds
  config      Manage Docker configs
  container   Manage containers
  context     Manage contexts
  engine      Manage the docker engine
  image       Manage images
  network     Manage networks
  node        Manage Swarm nodes
  plugin      Manage plugins
  secret      Manage Docker secrets
  service     Manage services
  stack       Manage Docker stacks
  swarm       Manage Swarm
  system      Manage Docker
  trust       Manage trust on Docker images
  volume      Manage volumes

Commands:
  attach      Attach local standard input, output, and error streams to a running container
  build       Build an image from a Dockerfile
  commit      Create a new image from a containers changes
  cp          Copy files/folders between a container and the local filesystem
  create      Create a new container
  diff        Inspect changes to files or directories on a containers filesystem
  events      Get real time events from the server
  exec        Run a command in a running container
  export      Export a containers filesystem as a tar archive
  history     Show the history of an image
  images      List images
  import      Import the contents from a tarball to create a filesystem image
  info        Display system-wide information
  inspect     Return low-level information on Docker objects
  kill        Kill one or more running containers
  load        Load an image from a tar archive or STDIN
  login       Log in to a Docker registry
  logout      Log out from a Docker registry
  logs        Fetch the logs of a container
  pause       Pause all processes within one or more containers
  port        List port mappings or a specific mapping for the container
  ps          List containers
  pull        Pull an image or a repository from a registry
  push        Push an image or a repository to a registry
  rename      Rename a container
  restart     Restart one or more containers
  rm          Remove one or more containers
  rmi         Remove one or more images
  run         Run a command in a new container
  save        Save one or more images to a tar archive (streamed to STDOUT by default)
  search      Search the Docker Hub for images
  start       Start one or more stopped containers
  stats       Display a live stream of container(s) resource usage statistics
  stop        Stop one or more running containers
  tag         Create a tag TARGET_IMAGE that refers to SOURCE_IMAGE
  top         Display the running processes of a container
  unpause     Unpause all processes within one or more containers
  update      Update configuration of one or more containers
  version     Show the Docker version information
  wait        Block until one or more containers stop, then print their exit codes

```

### Дополнительные материалы:

* [Docker 101](https://www.docker.com/101-tutorial)
* [Docker docks](https://docs.docker.com/get-started/overview/)
* Про задание ограничений можно почитать [здесь](https://docs.docker.com/config/containers/resource_constraints/)
* [Полное практическое руководство по Docker: с нуля до кластера на AWS](https://habr.com/ru/post/310460/)
* [Докер на русском](https://dker.ru/).